**Import Libraries**

In [39]:
import pandas as pd

**Load Data**

In [40]:
df = pd.read_csv('./100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


**Tokenize Function**

In [41]:
# TOKENIZE
def tokenize(text):
    text = text.lower()
    text = text.replace('?', '')
    text = text.replace("'", "")
    return text.split()

**Tokenize Example**

In [42]:
tokenize("Who wrote 'To Kill a Mockingbird'?")

['who', 'wrote', 'to', 'kill', 'a', 'mockingbird']

**Initialize Vocabulary**

In [43]:
# VOCAB
vocab = {'<UNK>': 0}

**Build Vocabulary Function**

In [44]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])
    
    merged_tokens = tokenized_question + tokenized_answer
    
    print(merged_tokens)

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

**Apply Build Vocabulary Function**

In [45]:
df.apply(build_vocab, axis=1)

['what', 'is', 'the', 'capital', 'of', 'france', 'paris']
['what', 'is', 'the', 'capital', 'of', 'germany', 'berlin']
['who', 'wrote', 'to', 'kill', 'a', 'mockingbird', 'harper-lee']
['what', 'is', 'the', 'largest', 'planet', 'in', 'our', 'solar', 'system', 'jupiter']
['what', 'is', 'the', 'boiling', 'point', 'of', 'water', 'in', 'celsius', '100']
['who', 'painted', 'the', 'mona', 'lisa', 'leonardo-da-vinci']
['what', 'is', 'the', 'square', 'root', 'of', '64', '8']
['what', 'is', 'the', 'chemical', 'symbol', 'for', 'gold', 'au']
['which', 'year', 'did', 'world', 'war', 'ii', 'end', '1945']
['what', 'is', 'the', 'longest', 'river', 'in', 'the', 'world', 'nile']
['what', 'is', 'the', 'capital', 'of', 'japan', 'tokyo']
['who', 'developed', 'the', 'theory', 'of', 'relativity', 'albert-einstein']
['what', 'is', 'the', 'freezing', 'point', 'of', 'water', 'in', 'fahrenheit', '32']
['which', 'planet', 'is', 'known', 'as', 'the', 'red', 'planet', 'mars']
['who', 'is', 'the', 'author', 'of', '19

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

**Vocabulary Size**

In [46]:
len(vocab)

324

**Accessing Vocabulary**

In [47]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

**Convert Text to Indices Function**

In [48]:
# CONVERTING WORDS TO NUMERICAL INDICES
def text_to_indices(text, vocab):

    indexed_text = []
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])

    return indexed_text

In [49]:
text_to_indices("What is a capital?", vocab)

[1, 2, 14, 4]

In [50]:
text_to_indices("What is a RNN?", vocab)

[1, 2, 14, 0]

---

In [51]:
import torch
from torch.utils.data import Dataset, DataLoader

In [52]:
class QADataset(Dataset):

    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab


    def __len__(self):
        return self.df.shape[0]


    def __getitem__(self, index):
        numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [53]:
dataset = QADataset(df, vocab)

In [54]:
dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True
)

**Dataloader Example**

In [55]:
for question, answer in dataloader:
    print(question, answer[0])

tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([205])
tensor([[ 1,  2,  3,  4,  5, 99]]) tensor([100])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([307])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([170])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([106])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([136])
tensor([[ 10,  96,   3, 104, 239]]) tensor([240])
tensor([[ 42, 250, 251, 118, 252, 253]]) tensor([254])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([116])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([154])
tensor([[ 10,   2,  62,  63,   3, 283,   5, 284]]) tensor([285])
tensor([[  1,   2,   3,   4,   5, 279]]) tensor([280])
tensor([[42, 43, 44, 45, 46, 47, 48]

**Simple RNN Model**

In [56]:
import torch.nn as nn

In [57]:
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
        self.rnn = nn.RNN(50, 64, batch_first=True)
        self.fc = nn.Linear(64, vocab_size)


    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))

        return output

**Model Architecture Test**

In [58]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)


a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)


b = x(a)
print("shape of b:", b.shape)


c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)


e = z(d.squeeze(0))
print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


**Learning Rate and Epochs**

In [59]:
learning_rate = 0.001

epochs = 50

**Initialize Model**

In [60]:
model = SimpleRNN(len(vocab))

**Loss Function and Optimizer**

In [61]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate
)

**Training Loop**

In [62]:
# TRAINING LOOP
for epoch in range(epochs):

    total_loss = 0

    for question, answer in dataloader:
        optimizer.zero_grad()

        # FORWARD PASS
        output = model(question)

        # LOSS -> OUTPUT SHAPE (1,324) - (1)
        loss = criterion(output, answer[0])

        # GRADIENTS
        loss.backward()

        # UPDATE
        optimizer.step()


        total_loss = total_loss + loss.item()

    print(f"Epoch: {epoch+1}, Total Loss: {total_loss:4f}")

Epoch: 1, Total Loss: 524.704813
Epoch: 2, Total Loss: 454.730257
Epoch: 3, Total Loss: 375.867527
Epoch: 4, Total Loss: 311.789190
Epoch: 5, Total Loss: 259.789129
Epoch: 6, Total Loss: 211.704488
Epoch: 7, Total Loss: 168.043764
Epoch: 8, Total Loss: 130.885290
Epoch: 9, Total Loss: 100.032030
Epoch: 10, Total Loss: 76.734492
Epoch: 11, Total Loss: 59.418766
Epoch: 12, Total Loss: 46.638134
Epoch: 13, Total Loss: 37.137787
Epoch: 14, Total Loss: 30.338483
Epoch: 15, Total Loss: 24.789986
Epoch: 16, Total Loss: 20.861416
Epoch: 17, Total Loss: 17.651503
Epoch: 18, Total Loss: 15.154345
Epoch: 19, Total Loss: 12.979251
Epoch: 20, Total Loss: 11.490686
Epoch: 21, Total Loss: 9.893620
Epoch: 22, Total Loss: 8.727756
Epoch: 23, Total Loss: 7.703395
Epoch: 24, Total Loss: 6.832693
Epoch: 25, Total Loss: 6.140806
Epoch: 26, Total Loss: 5.571400
Epoch: 27, Total Loss: 5.004740
Epoch: 28, Total Loss: 4.554403
Epoch: 29, Total Loss: 4.162269
Epoch: 30, Total Loss: 3.808853
Epoch: 31, Total Los

**Predict Function**

In [63]:
def predict(model, question, threshold=0.5):

    # CONVERT QUESTION TO NUMBERS
    numerical_question = text_to_indices(question, vocab)

    # TENSOR
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # SEND TO MODEL
    output = model(question_tensor)

    # CONVERT LOGITS TO PROBS
    probs = torch.nn.functional.softmax(output, dim=1)

    # FIND INDEX OF MAX PROB
    value, index = torch.max(probs, dim=1)


    if value < threshold:
        print("I don't know")
    else:
        print(list(vocab.keys())[index])

**Predict Function Example**

In [64]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [65]:
predict(model, 'What is the capital of France?')

paris


In [66]:
# INTRODUCING SMALL BIT OF VARIATION TO THE QUESTION:
predict(model, 'WHAT IS CAPITAL CITY OF FRANCE?')

predict(model, 'What is the capital of a european country named France?')

paris
I don't know


In [67]:
# QUESTION WHICH MODEL HAS NOT SEEN EVEN ONCE:

predict(model, 'Who is Avinash Yadav?')

I don't know
